<a href="https://colab.research.google.com/github/mz1119/cup-classification-model/blob/main/Applying_Classification_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd

milkWater_model = torch.load("/content/drive/My Drive/Blue Hill Cup Images/Milk_and_Water_Classification_Saved_Model.pth")
espresso_model = torch.load("/content/drive/My Drive/Blue Hill Cup Images/Espresso_Classification_Saved_Model.pth")
latte_model = torch.load("/content/drive/My Drive/Blue Hill Cup Images/Latte_Classification_Saved_Model.pth")
coffee_model = torch.load("/content/drive/My Drive/Blue Hill Cup Images/Coffee_Classification_Saved_Model.pth")
detect_model = torch.load("/content/drive/My Drive/Blue Hill Cup Images/Saved_Cup_Model.pth")

In [3]:
def classifyImage(file_name, device, drink_type):
    with Image.open("/content/drive/My Drive/Blue Hill Cup Images/Cup_Image_Dataset/" + file_name) as img:
        load_variable = img.load()
        img.convert("RGB")
    #convert img to tensor
    PIL_to_tensor = transforms.Compose([
            transforms.ToTensor()
            ])
    img = PIL_to_tensor(img)
    
    # put the model in evaluation mode
    detect_model.eval()
    with torch.no_grad():
        prediction = detect_model([img.to(device)])

    #get images and mask
    test_img = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
    mask_img = Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

    #make black pixels on mask transparent
    mask_img = mask_img.convert("RGBA")
    datas = mask_img.getdata()
    newData = []
    for item in datas:
        if item[0] <= 100 and item[1] <= 100 and item[2] <= 100:
            newData.append((0, 0, 0, 255)) #black
        elif item[0] == item[1] == item[2]:
            newData.append((255, 255, 255, 0)) #transparent
        else:
            newData.append(item)
    mask_img.putdata(newData)

    #overlay partial transparent mask_img over test_img
    test_img.paste(mask_img, (0, 0), mask_img)

    #get shape of bounding box
    bbox_shape = []
    for counter in range(prediction[0]['boxes'][0].size()[0]):
      bbox_shape.append(prediction[0]['boxes'][0][counter].item())
      
    #crop image
    test_img = test_img.crop(tuple(bbox_shape))
    
    #show image
    # imshow(np.array(test_img))


    #do some transforms on image
    transform = transforms.Compose(
            [
                transforms.Resize((366,366)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])
            ]
        )
    test_img = transform(test_img)
    test_img = test_img.unsqueeze(0)

    # def imshow(img):
    #     img = img / 2 + 0.5     # unnormalize
    #     npimg = img.numpy()
    #     plt.imshow(np.transpose(npimg, (1, 2, 0)))
    #     plt.show()
    # imshow(torchvision.utils.make_grid(test_img))

    #figure out which model to use
    reference_dict = {'coffee': coffee_model, 'latte': latte_model, 'espresso': espresso_model,'milkWater': milkWater_model}
    classifier = reference_dict[drink_type]

    classifier.eval()
    with torch.no_grad():
        outputs = classifier(test_img.to(device))
    _, predicted = torch.max(outputs,1)

    classes = ('no_topping','has_topping','has_ice','ice_and_topping')

    return classes[predicted[0]]

In [5]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#get device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

file_name = '20210626_144211_Image.jpeg'

classifyImage(file_name = file_name, device = device, drink_type = 'milkWater')

cuda


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


'ice_and_topping'

In [10]:
#Milk and Water
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
data_frame = pd.read_json('/content/drive/My Drive/Blue Hill Cup Images/MilkWaterJSON.json')

successfully_classified = 0

for row in range(data_frame.shape[0]):
    print(row)
    file_name = data_frame.iloc[row, 0]
    if classifyImage(file_name = file_name, device = device, drink_type = 'milkWater') == data_frame.iloc[row, 1]:
        successfully_classified += 1
    else:
        print(classifyImage(file_name = file_name, device = device, drink_type = 'milkWater'), data_frame.iloc[row, 1])

print('correct ratio: ' + str(successfully_classified/data_frame.shape[0]))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#Espresso
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
data_frame = pd.read_json('/content/drive/My Drive/Blue Hill Cup Images/EspressoJSON.json')

successfully_classified = 0

for row in range(data_frame.shape[0]):
    print(row)
    file_name = data_frame.iloc[row, 0]
    if classifyImage(file_name = file_name, device = device, drink_type = 'espresso') == data_frame.iloc[row, 1]:
        successfully_classified += 1
    else:
        print(classifyImage(file_name = file_name, device = device, drink_type = 'espresso'), data_frame.iloc[row, 1])

print('correct ratio: ' + str(successfully_classified/data_frame.shape[0]))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
no_topping has_topping
19
20
21
22
23
24
25
no_topping has_topping
26
27
28
29
30
no_topping has_topping
31
32
no_topping has_topping
33
no_topping has_topping
34
no_topping has_topping
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [8]:
#Latte
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
data_frame = pd.read_json('/content/drive/My Drive/Blue Hill Cup Images/LatteJSON.json')

successfully_classified = 0

for row in range(data_frame.shape[0]):
    print(row)
    file_name = data_frame.iloc[row, 0]
    if classifyImage(file_name = file_name, device = device, drink_type = 'latte') == data_frame.iloc[row, 1]:
        successfully_classified += 1
    else:
        print(classifyImage(file_name = file_name, device = device, drink_type = 'latte'), data_frame.iloc[row, 1])

print('correct ratio: ' + str(successfully_classified/data_frame.shape[0]))


0
1
2


KeyboardInterrupt: ignored

In [9]:
#Coffee
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
data_frame = pd.read_json('/content/drive/My Drive/Blue Hill Cup Images/CoffeeJSON.json')

successfully_classified = 0

for row in range(data_frame.shape[0]):
    print(row)
    file_name = data_frame.iloc[row, 0]
    if classifyImage(file_name = file_name, device = device, drink_type = 'coffee') == data_frame.iloc[row, 1]:
        successfully_classified += 1
    else:
        print(classifyImage(file_name = file_name, device = device, drink_type = 'coffee'), data_frame.iloc[row, 1])

print('correct ratio: ' + str(successfully_classified/data_frame.shape[0]))


0
1
2


KeyboardInterrupt: ignored